### RS
subreddit | author | hidden | link_flair_css_class | id | is_self | edited | selftext | score | retrieved_on | domain | brand_safe | contest_mode | media | secure_media_embed | created_utc | archived | distinguished | secure_media | locked | thumbnail | url | link_flair_text | over_18 | author_flair_text | suggested_sort | quarantine | subreddit_id | spoiler | stickied | hide_score | author_flair_css_class | permalink | media_embed | title | gilded | num_comments | 

### RC
author | gilded | subreddit | created_utc | edited | controversiality | author_flair_text | parent_id | score | link_id | id | author_flair_css_class | distinguished | stickied | subreddit_id | body | retrieved_on |

#### API: Fullname Prefix

- t1_	Comment
- t2_	Account
- t3_	Link
- t4_	Message
- t5_	Subreddit
- t6_	Award

In [28]:
import bz2, json
from collections import namedtuple

In [120]:
allowed_comment_vars = ['author', 'body', 'score', 'parent_id', 'id', 'link_id']
CommentStruct = namedtuple('CommentStruct', ' '.join(allowed_comment_vars))

class Post:
    def __init__(self, **kwargs):
        self.url = kwargs['url']
        self.author = kwargs['author']
        self.subreddit = kwargs['subreddit']
        self.score = kwargs['score']
        self.selftext = kwargs['selftext']
        self.score = kwargs['score']
        self.id = kwargs['id']
        self.num_comments = kwargs['num_comments']
        self.comments = {}
    
    # Maintain a tree structure for comments
    # Root-to-leaf paths using a DFS traversal 
    #  would give data samples for individual conversations
    def add_comment(self, **kwargs):
        c = CommentStruct(**{key:var for key, var in kwargs.items() if key in allowed_comment_vars})
        self.comments['t1_' + c.id] = c
    
    def __str__(self):
        s = ""
        for key, val in self.__dict__.items():
            if key != 'comments':
                s += f"{key}:\t{val}\n"
            else:
                s += "\nComments: \n"
                for k,c in val.items():
                    s += f"{c}\n\n"
        return s
    
    def __repr__(self):
        return self.__str__()

In [121]:
# accepted_subreddits = ['AskReddit', 'gifts', 'depression']
accepted_subreddits = ['AskReddit', 'gifts', 'depression']

In [122]:
posts = {}

with bz2.BZ2File('./data/RS_2017-03.bz2', 'r') as fp:
    for i, line in enumerate(fp):
        if i == 10000:
            break
        curr_post = json.loads(line)
        if 'subreddit' in curr_post:
            if (curr_post['subreddit'] == 'AskReddit') and (curr_post['num_comments'] >= 3):
                print(f'{i}', end='\r')
                # posts.append(curr_post)
                posts['t3_'+curr_post['id']] = Post(**curr_post)

In [123]:
all_comments = []

In [124]:
with bz2.BZ2File('./data/RC_2017-03.bz2', 'r') as fp:
    for i, line in enumerate(fp):
        if i == 10000:
            break
        print(f'{i}', end='\r')
        curr_comment = json.loads(line)
        if (curr_comment['subreddit'] == 'AskReddit'):
            all_comments.append(curr_comment)
            if curr_comment['link_id'] in posts:
                posts[curr_comment['link_id']].add_comment(**curr_comment)
        # link = f'https://www.reddit.com/r/{curr_comment["subreddit"]}/comments/{curr_comment["link_id"].split("_")[1]}'        

In [125]:
matched_ids = [p.id for t3_id, p in posts.items() if len(p.comments) > 0]

In [126]:
posts['t3_5wrk9g']

url:	https://www.reddit.com/r/AskReddit/comments/5wrk9g/if_public_schools_no_longer_received_funding_from/
author:	Jsreb
subreddit:	AskReddit
score:	5
selftext:	[removed]
id:	5wrk9g
num_comments:	12

Comments: 
CommentStruct(author='man-eating-chicken', body='other taxes', score=2, parent_id='t3_5wrk9g', id='decf3n7', link_id='t3_5wrk9g')

CommentStruct(author='Aw_Frig', body='child slave labor? ', score=3, parent_id='t3_5wrk9g', id='decf3ps', link_id='t3_5wrk9g')

CommentStruct(author='PatacusX', body='Tuition, or moving all prisons into the basements and having the prisoners run n hamster wheels all day to generate power.', score=2, parent_id='t3_5wrk9g', id='decf6le', link_id='t3_5wrk9g')
